<a href="https://colab.research.google.com/github/Mwadz/Sematic-Text-Similarity/blob/main/Semantic_text_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets
! pip install sentence_transformers


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 306.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 342.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 311.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 224.0 kB/s eta 0:00:0000:0100:03
  Using cached huggingface_hub-0.9.1-py3-none-any.whl (120 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 375.6 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.0/575.0 kB 368.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 244.9 kB/s eta 0:00:00a 0:00:01
  Using cached requests-2.28.1-py3-none-any.w

In [3]:
#loading training set
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
from datasets import load_dataset


# Load the English STSB dataset
stsb_dataset = load_dataset('stsb_multi_mt', 'en')
stsb_train = pd.DataFrame(stsb_dataset['train'])
stsb_test = pd.DataFrame(stsb_dataset['test'])

# Check loaded data
print(stsb_train.shape, stsb_test.shape)
stsb_test.head()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset stsb_multi_mt downloaded and prepared to /Users/steveogaja/.cache/huggingface/datasets/stsb_multi_mt/en/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 30.81it/s]


(5749, 3) (1379, 3)


,sentence1,sentence2,similarity_score
0,A girl is styling her hair.,A girl is brushing her hair.,2.5
1,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.,3.6
2,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.,5.0
3,A man is cutting up a cucumber.,A man is slicing a cucumber.,4.2
4,A man is playing a harp.,A man is playing a keyboard.,1.5


## Creating helper functions
* The first function is to pre-process texts by lemmatizing, lowercasing, and removing numbers and stop words.
* The second function takes in two columns of text embeddings and returns the row-wise cosine similarity between the two columns.

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import spacy
nlp = spacy.load("en_core_web_sm")

def text_processing(sentence):
    """
    Lemmatize, lowercase, remove numbers and stop words
    
    Args:
      sentence: The sentence we want to process.
    
    Returns:
      A list of processed words
    """
    sentence = [token.lemma_.lower()
                for token in nlp(sentence) 
                if token.is_alpha and not token.is_stop]
    
    return sentence


def cos_sim(sentence1_emb, sentence2_emb):
    """
    Cosine similarity between two columns of sentence embeddings
    
    Args:
      sentence1_emb: sentence1 embedding column
      sentence2_emb: sentence2 embedding column
    
    Returns:
      The row-wise cosine similarity between the two columns.
      For instance is sentence1_emb=[a,b,c] and sentence2_emb=[x,y,z]
      Then the result is [cosine_similarity(a,x), cosine_similarity(b,y), cosine_similarity(c,z)]
    """
    cos_sim = cosine_similarity(sentence1_emb, sentence2_emb)
    return np.diag(cos_sim)

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [5]:
data = (pd.read_csv("/content/SBERT_data.csv")).drop(['Unnamed: 0'], axis = 1)

prompt = input("Enter prompt: ")
data['prompt']= prompt
data.rename(columns = {'target_text':'sentence2', 'prompt':'sentence1'}, inplace = True)
data['sentence2'] = data['sentence2'].astype('str')
data['sentence1']  = data['sentence1'].astype('str')

data.head()

Enter prompt: pecifies the Loan Program that exempts the transaction from Regulation Z Ability-to-Repay requirements


,input_text,sentence2,sentence1
0,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,When true indicates the purpose of the extensi...,pecifies the Loan Program that exempts the tra...
1,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,Specifies the creditor organization type exemp...,pecifies the Loan Program that exempts the tra...
2,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,A free-form text field used to collect additio...,pecifies the Loan Program that exempts the tra...
3,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,Specifies the Loan Program that exempts the tr...,pecifies the Loan Program that exempts the tra...
4,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,A free-form text field used to collect additio...,pecifies the Loan Program that exempts the tra...


In [6]:
from sentence_transformers import CrossEncoder
XpathFinder = CrossEncoder("cross-encoder/stsb-roberta-base")
sentence_pairs = []
for sentence1, sentence2 in zip(data['sentence1'],data['sentence2']):
  sentence_pairs.append([sentence1, sentence2])

data['SBERT CrossEncoder_Score'] = XpathFinder.predict(sentence_pairs, show_progress_bar = True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/142 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Batches:   0%|          | 0/681 [00:00<?, ?it/s]

In [7]:
data.sort_values(by=['SBERT CrossEncoder_Score'], ascending=False)


,input_text,sentence2,sentence1,SBERT CrossEncoder_Score
5,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,Specifies the reason that transaction is exemp...,pecifies the Loan Program that exempts the tra...,0.838702
3,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,Specifies the Loan Program that exempts the tr...,pecifies the Loan Program that exempts the tra...,0.791737
1,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,Specifies the creditor organization type exemp...,pecifies the Loan Program that exempts the tra...,0.784762
8,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,Specifies the method by which the creditor sat...,pecifies the Loan Program that exempts the tra...,0.734334
0,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,When true indicates the purpose of the extensi...,pecifies the Loan Program that exempts the tra...,0.710616
...,...,...,...,...
6093,DOCUMENT/SIGNATORIES/SIGNATORY/EXECUTION/ADDRE...,The two-character representation of the countr...,pecifies the Loan Program that exempts the tra...,0.002239
17657,DOCUMENT/DEAL_SETS/DEAL_SET/PARTIES/PARTY/ADDR...,A unique alphanumeric string identifying the r...,pecifies the Loan Program that exempts the tra...,0.002236
17658,DOCUMENT/PARTIES/PARTY/ADDRESSES/ADDRESS/Rural...,A unique alphanumeric string identifying the r...,pecifies the Loan Program that exempts the tra...,0.002232
6133,DOCUMENT/DEAL_SETS/DEAL_SET/PARTIES/PARTY/ADDR...,The name of the country which belongs to an ad...,pecifies the Loan Program that exempts the tra...,0.002232


In [ ]:
import pickle

filename = 'XpathFinder1.sav'
pickle.dump(XpathFinder, open(filename, 'wb'))